<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Federated averaging function
def federated_averaging(global_model, client_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k] for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)

# Simulate federated learning
def federated_learning(num_clients=5, num_rounds=10):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    global_model = Net()
    client_models = [Net() for _ in range(num_clients)]
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

    for round in range(num_rounds):
        for client_model in client_models:
            optimizer = optim.SGD(client_model.parameters(), lr=0.01)
            for images, labels in train_loader:
                images, labels = images.to('cpu'), labels.to('cpu')  # Ensure data is on CPU
                client_model.train()
                optimizer.zero_grad()
                output = client_model(images)
                loss = nn.CrossEntropyLoss()(output, labels)
                loss.backward()
                optimizer.step()

        federated_averaging(global_model, client_models)
        print(f'Round {round + 1} complete')

    return global_model

global_model = federated_learning()